# OpenStreetMap Data Case Study

## Map Area
https://mapzen.com/data/metro-extracts/metro/chicago_illinois/

### 0. Sampling 数据采集

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

#get sample element

import xml.etree.ElementTree as ET 

OSM_FILE = "chicago_illinois.osm"
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element
#!/usr/bin/env python
# -*- coding: utf-8 -*-


def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="utf-8"?>\n')
    output.write(b'<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')

### 1. Counting the numbers of unique tags

In [2]:
import pprint
import xml.etree.ElementTree as ET

def count_tags(filename):
    tags={}
    for event,elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag]+=1
        else:
            tags[elem.tag]=1
    return tags

tags=count_tags('sample.osm')
pprint.pprint(tags)

{'member': 8350,
 'nd': 1053477,
 'node': 880236,
 'osm': 1,
 'relation': 476,
 'tag': 683950,
 'way': 125150}


### 1.5 explore the tags to find problems

In [3]:
import xml.etree.cElementTree as ET
import pprint
import re

from collections import defaultdict

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

OSMFILE = "sample.osm"

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(element.attrib['k']):
                keys['lower'] = keys['lower'] + 1
            elif lower_colon.search(element.attrib['k']):
                keys['lower_colon'] = keys['lower_colon'] + 1
            elif problemchars.search(element.attrib['k']):
                keys['problemchars'] = keys['problemchars'] + 1
            else:
                keys['other'] = keys['other'] + 1
    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

pprint.pprint(process_map(OSMFILE))

{'lower': 211501, 'lower_colon': 312727, 'other': 159721, 'problemchars': 1}


### 2. data audit 

In [4]:
# figure out the dirty data (especially the street type data)
# print out the new name and use update_name function at next cell ( to build csv files)

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE_sample = "sample.osm"
regex = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard","Bouevard", "Broadway","Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","B","Blvd","C","Circle","Center","D","E","F","G","H","Highway","J","K","L","Larchmont",
           "M","Market","Millsfell","N","North","O","Park","Path","Plaza","Row","South","Terrace",
           "Walk","Way","West","mall"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Rd": "Road",
            "raod": "road",
            "Ln": "Lane",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Ave.":"Avenue",
            "Trl":"Trail",
            "Cir":"Circle",
            "202":"202 Avenue",
            "231":"2312 Avenue",
            "38":"38 Avenue",
            "3EF":"3EF Avenue",
            "47":"47 Avenue",
            "53":"53 Avenue",
            "59":"59 Avenue",
            "6":"6 Avenue",
            "64":"64 Avenue",
            "8":"8 Avenue",
            "64":"64 Avenue",
            "Avenu":"Avenue",
            "Ct":"Court",
            "Dr":"Drive",
            "Dr.":"Drive",
            "Harrison":"Harrison Street"
            }


# Search string for the regex. If it is matched and not in the expected list then add this as a key to the set.
def audit_street(street_types, street_name): 
    m = regex.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem): # Check if it is a street name
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile): # return the list that satify the above two functions
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street(street_types, tag.attrib['v'])

    return street_types

pprint.pprint(dict(audit(OSMFILE_sample))) # print the existing names

def string_case(s):     # change string into titleCase except for UpperCase
    if s.isupper():
        return s
    else:
        return s.title()

# return the updated names
def update_name(name, mapping):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            name = re.sub(regex, mapping[street_type], name)

    return name

update_street = audit(OSMFILE_sample) 

# print the updated names
for street_type, ways in update_street.items():
    for name in ways:
        better_name = update_name(name, mapping)
        print (name, "=>", better_name) 

{'202': set(['S Northwest Hwy #202']),
 '231': set(['East US 231']),
 '38': set(['Route 38']),
 '3EF': set(['N Lake Shore Dr #3EF']),
 '47': set(['Illinois Route 47']),
 '53': set(['Route 53']),
 '59': set(['Route 59', 'S IL State Route 59']),
 '6': set(['US Highway 6']),
 '64': set(['IL Route 64']),
 '8': set(['IL-59 #8']),
 'Ave': set(['Alabama Ave',
             'Arkansas Ave',
             'California Ave',
             'Central Ave',
             'Dundee Ave',
             'Hawaii Ave',
             'Independence Ave',
             'Lake Ave',
             'Larkin Ave',
             'N Wheaton Ave',
             'N. Glenwood Ave',
             'New York Ave',
             'North Milwaukee Ave',
             'Ogden Ave',
             'S Western Ave',
             'S. Gary Ave',
             'South Lake Park Ave',
             'W Belmont Ave']),
 'Ave.': set(['Milwaukee Ave.',
              'W Park Ave.',
              'W. Euclid Ave.',
              'Waiola Ave.']),
 'Avenu': set([

### 3. build CSV files from OSM and also parse, clean and shape data

In [6]:
########## Don't forget to use update_name function declared above
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from unittest import TestCase

import cerberus
import schema

OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                if is_street_name(child):
                    node_tag['value'] = update_name(child.attrib['v'], mapping)
### important! update the correct name!!!!!!
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                if is_street_name(child):
                    node_tag['value'] = update_name(child.attrib['v'], mapping)
### important! update the correct name!!!!!!
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    if is_street_name(child):
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    if is_street_name(child):
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

            
def validate_element(element, validator, schema=SCHEMA):
    #Raise ValidationError if element does not match schema
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)

ImportError: No module named cerberus

### 4. create database of the CSV files

In [ ]:
import csv, sqlite3

con = sqlite3.connect("chicago_illinois.db")
con.text_factory = str
cur = con.cursor()

# create nodes table

cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rt',encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['b\'id\''], i['b\'lat\''], i['b\'lon\''], i['b\'user\''], i['b\'uid\''], i['b\'version\''], i['b\'changeset\''], i['b\'timestamp\'']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#create nodes_tags table

cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','rt',encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['b\'id\''], i['b\'key\''], i['b\'value\''], i['b\'type\'']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

#Create ways table

cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','rt',encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['b\'id\''], i['b\'user\''], i['b\'uid\''], i['b\'version\''], i['b\'changeset\''], i['b\'timestamp\'']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#Create ways_nodes table\

cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','rt',encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['b\'id\''], i['b\'node_id\''], i['b\'position\'']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
con.commit()

#Create ways_tags table

cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
with open('ways_tags.csv','rt',encoding = 'utf-8') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['b\'id\''], i['b\'key\''], i['b\'value\''], i['b\'type\'']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

### 5. query
>nodes<br>ways<br>unique_users<br>top_contributing_users<br> amount of starbucks

In [ ]:
import csv, sqlite3

def number_of_nodes(): # 
	result = cur.execute('SELECT COUNT(*) FROM nodes')
	return result.fetchone()[0]

def number_of_ways():
	result = cur.execute('SELECT COUNT(*) FROM ways')
	return result.fetchone()[0]

def number_of_unique_users():
	result = cur.execute('SELECT COUNT(DISTINCT(e.uid)) \
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')
	return result.fetchone()[0]
    
def top_contributing_users():
	users = []
	for row in cur.execute('SELECT e.user, COUNT(*) as num \
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
            GROUP BY e.user \
            ORDER BY num DESC \
            LIMIT 10'):
		users.append(row)
	return users

def number_of_users_contributing_once():
	result = cur.execute('SELECT COUNT(*) \
            FROM \
                (SELECT e.user, COUNT(*) as num \
                 FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
                 GROUP BY e.user \
                 HAVING num=1) u')
	return result.fetchone()[0]


if __name__ == '__main__':
	
	con = sqlite3.connect("chicago_illinois.db")
	cur = con.cursor()
	
	print ("Number of nodes: " , number_of_nodes())
	print ("Number of ways: " , number_of_ways())
	print ("Number of unique users: " , number_of_unique_users())
	print ("Top contributing users: " , top_contributing_users())
	print ("Number of users contributing once: " , number_of_users_contributing_once())


### 5. Starbucks~

In [ ]:
cur.execute("SELECT COUNT(*) FROM nodes_tags WHERE value LIKE '%Starbucks%';")
print(cur.fetchall())

# 6. 结论
## 芝加哥的地图集很大，所以说错误数据很多，在清洗数据的时候需要分很多类,另外芝加哥城市的街道类型命名也多种多样，这也使得数据集具有很大的复杂度

# 7. 更多建议
## 在提供者提供数据并提交的时候是否可以直接去自动规范输入数据的类型呢？或者在OpenMap的网站提供一个提交数据的准则，这样可以提高数据分析的效率